# Caso 1: Open-Meteo
En este caso práctico vamos a obtener datos meteorológicos desde una API pública (Open-Meteo). El proyecto tendrá los siguientes componentes:
- Productor kafka. Programa Python que accederá a la API a través del módulo request y producirá los resultados a un topic que será accedido después por nuestro programa spark.
- Programa spark. programa que recopilará los datos de kafka y realizará agregaciones por ventanas de tiempo. En función de adónde se envíen los resultados tendremos dos variantes:
  - A consola.
  - A archivo parquet.
- Por último tendremos un programa echo en *streamlit* que mostrará de forma visual los datos recopilados.
## productor_open_meteo.py
Necesitamos las siguientes importaciones:
- **KafkaProducer** del módulo **kafka**.
- **dumps** del paquete **json**-
- **requests**: Para realizar peticiones a la API.
- **time**.
- **datetime**.

Los pasos a seguir serán los siguientes:
1. Crear un productor kafka.
2. Hacer una llamada a la API de Open-Meteo.
3. Transformar los datos a json y formatearlos.
4. Usar el método send() de la clase *KafkaProducer* para escribir los datos a un topic.

Lo primero será crear el productor kafka indicando el **bootstrap-server** y la codificación.

In [ ]:
from kafka import KafkaProducer
producer = KafkaProducer(
    bootstrap_servers=['kafka-1:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

Vamos a emplear la API de Open-Meteo. El primer paso sería consultar la [documentación oficial](https://open-meteo.com/en/docs). En la sección **API Documentation* vemos un listado de los parámetros disponibles, ente ellos *latitude* y *longitude* que son obligatorios.

Vemos que las variables están distribuidas en varios gurpos: daily weather, hourly weather, current weather... Nos interesa este último.

Para comunicarnos con la API vamos a emplear el módulo **requests**. Especificamos la url de la api y los parámetros (pares clave-valor) y llamamos al método **get()**. Convertimos la respuesta a *json* para explorar su estructura:

In [1]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 42.33669,        # Coordenadas de Vigo
    "longitude": -7.86407,
    "current_weather": "true"
}

response = requests.get(url, params=params)

if response.ok:
    data = response.json()
    print(json.dumps(data, indent=2))
else:
    print("Erro:", response.status_code)

{
  "latitude": 42.3125,
  "longitude": -7.875,
  "generationtime_ms": 0.046372413635253906,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 147.0,
  "current_weather_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature": "\u00b0C",
    "windspeed": "km/h",
    "winddirection": "\u00b0",
    "is_day": "",
    "weathercode": "wmo code"
  },
  "current_weather": {
    "time": "2025-05-05T14:30",
    "interval": 900,
    "temperature": 19.0,
    "windspeed": 15.7,
    "winddirection": 11,
    "is_day": 1,
    "weathercode": 3
  }
}


Una vez tenemos la respuesta el siguiente paso es obtener los datos que nos interesan, en nuestro caso *current_weather*. Le añadimos el nombre de la ciudad y un timestamp normalizado

In [2]:


from datetime import datetime
data = response.json()["current_weather"]

data["city"]="Ourense"
data["timestamp"] = datetime.utcnow().isoformat()
print(data)

{'time': '2025-05-05T14:30', 'interval': 900, 'temperature': 19.0, 'windspeed': 15.7, 'winddirection': 11, 'is_day': 1, 'weathercode': 3, 'city': 'Ourense', 'timestamp': '2025-05-05T14:43:19.547496'}


Una vez tengamos los datos en el formato que queremos almacenados en una variable (en el ejemplo *data*) los escribimos al topic indicado:

In [ ]:
from kafka import KafkaProducer
#...

producer.send("topic",data)

Se puede hacer lo mismo con varias localizaciones empleando un bucle.
## Programa Spark
La estructura del programa *spark* es muy similar a los notebooks vistos previamente:
1. Inicializamos la sesión spark: Tenos que incluir los paquetes necesarios para usar *Kafka*.
2. Definimos el esquema del DataFrame inicial.
3. Definimos el flujo de entrada. En este caso tenemos que indicar, entre otros, el **bootstrap-server** y el nombre del **topic** del que se leerán los datos.
4. Creamos un *DataFrame* nuevo transformando los datos iniciales. Hay que tener en cuenta las limitaciones para distintos tipos de agregaciones (especificadas en el documento 00 y en la [documentación oficial](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).
5. Iniciamos el procesamiento, como siempre, indicando el destino y el modo.

Ejemplo de inicialización de la sesión spark:


In [ ]:
### EJEMPLO: NO EJECUTAR
spark = SparkSession.builder \
    .appName("OpenMeteoStreamingClean") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

Ejemplo de definición del esquema:

In [ ]:
### EJEMPLO: NO EJECUTAR
schema = StructType() \
    .add("temperature", DoubleType()) \
    .add("windspeed", DoubleType()) \
    .add("winddirection", DoubleType()) \
    .add("weathercode", IntegerType()) \
    .add("time", StringType()) \
    .add("city", StringType()) \
    .add("local_timestamp", StringType())

Ejemplo De definición del flujo de entrada:

In [ ]:
### EJEMPLO: NO EJECUTAR
# Ejemplo Open-Meteo
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-1:9092") \
    .option("subscribe", "open-meteo-weather") \
    .option("startingOffsets", "latest") \
    .load()

# Ejemplo general
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "servidor-kafka") \
    .option("subscribe", "nombre-topic") \
    .option("startingOffsets", "latest") \
    .load()


Ejemplo transformación de datos:

In [ ]:
### EJEMPLO: NO EJECUTAR

# Transformación simple: filtrado y limpieza.
df_parsed = df_kafka.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select(
        col("data.city"),
        col("data.temperature"),
        col("data.windspeed"),
        col("data.winddirection"),
        to_timestamp(col("data.local_timestamp")).alias("event_time")
    )

#Transformación adicional empleando windowing y watermarking:
# Agrupación por ventana y ciudad
df_grouped = df_parsed \
    .withWatermark("event_time", "2 minutes") \
    .groupBy(
        window(col("event_time"), "1 minute"),
        col("city")
    ).agg(
        avg("temperature").alias("avg_temp"),
        avg("windspeed").alias("avg_wind")
    )


Ejemplo de inicio del procesamiento en streaming:

In [ ]:
### EJEMPLO: NO EJECUTAR

# Mostrar en consola
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
### EJEMPLO: NO EJECUTAR

# Almacenar en parquet:
query=df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "hdfs://spark-master:9000/user/jovyan/weather_aggregated/") \
    .option("checkpointLocation", "hdfs://spark-master:9000/user/jovyan/checkpoint_weather_aggregated/") \
    .start()

En esta carpeta hay ejemplos de productor y de programa spark. Recordad que para que funcione al ejecutarlos con spark submit hay que añadir la siguiente opción: **--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0**